In [10]:
import serial
import re
import time
import pygame
import sys
import math

In [11]:
class Position:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0

class Orientation:
    def __init__(self):
        self.roll = 0
        self.pitch = 0

class Pose:
    def __init__(self):
        self.position = Position()
        self.orientation = Orientation()

    def update_pose(self, pose_array):
        self.position.x = pose_array[0]
        self.position.y = pose_array[1]
        self.position.z = pose_array[2]
        self.orientation.pitch = pose_array[3]
        self.orientation.roll = pose_array[4]

    def __str__(self):
        return f"X: {self.position.x}, Y: {self.position.y}, Z: {self.position.z}, P: {self.orientation.pitch}, R: {self.orientation.roll}"

In [117]:
import serial
import re
import time

from Pose import Pose

class Robot:
    def __init__(self, port):
        self.ser = serial.Serial(port)
        self.ser.write(b"CON \r")
        time.sleep(5)
        # self.ser.write(b"HOME \r")
        # time.sleep(160)
        response = self.ser.read_all()
        print(response.decode('ascii'))

        self.pose = Pose()  

        self.get_intial_position()

    def close(self):
        self.ser.close()

    def get_intial_position(self):
        self.ser.write(b"HERE P1 \r")
        time.sleep(1)
        self.ser.write(b"LISTPV P1 \r")
        time.sleep(1)
        response = self.ser.read_all()
        print(response)
        pattern = "X:([\s-]\d+)\s*Y:([\s-]\d+)\s*Z:([\s-]\d+)\s*P:([\s-]\d+)\s*R:([\s-]\d+)"
        # X: 3423, Y: -353, Z: 3423, P: -201, R: 76
        match = re.search(pattern, response.decode('ascii'))
        self.pose.update_pose([
            int(match.group(1)),
            int(match.group(2)),
            int(match.group(3)),
            int(match.group(4)),
            int(match.group(5))
        ])

    
    '''
    Starts from the initial position and updates the position
    '''

    def move_robot(self, increment, delay, delay2):
        if not (5000 < self.pose.position.z < 7000):
            print("not a valid movement")
            pass
        self.pose.update_pose([
            self.pose.position.x,
            self.pose.position.y,
            self.pose.position.z + increment,
            self.pose.orientation.pitch,
            self.pose.orientation.roll,
        ])
        
        print(self.ser.read_all())
        position_command = f"SETPVC P1 Z {self.pose.position.z} \r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        check = ''
        while True:
            check += self.ser.read_all().decode('ascii')
            print(check)
            if bool(re.search("Done.", check)):
                break
            time.sleep(delay)
        position_command = f"MOVEL P1 {delay2}\r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        check = ''
        while True:
            check += self.ser.read_all().decode('ascii')
            print(check)
            if bool(re.search("Done.", check)):
                break
            time.sleep(delay)

    def print_pose(self):
        print(self.pose)

In [118]:
robot = Robot("COM5")

CON 
CONTROL ENABLED.
>
b'HERE P1 \r\n\x00Done.\r\n>LISTPV P1 \r\nPosition P1\r\n 1: 1       2:-515     3:-823     4: 232     5: 392    \r\n X: 7469    Y:-352     Z: 6522    P: 73      R:-165    \r\n>'


In [125]:
pygame.init()
pygame.joystick.init()

joystick = pygame.joystick.Joystick(0)
joystick.init()

running = True

r1_pressed = False
l1_pressed = False

while running:
    start_time = time.time()
    if (r1_pressed):
        robot.move_robot_z(20, 0.1, 50)
    
    if (l1_pressed):
        robot.move_robot_z(-20, 0.1, 50)
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            robot.close()
            pygame.quit()

        if event.type == pygame.JOYBUTTONDOWN:
            # print(event)
            if event.button == 4:
                l1_pressed = True
            if event.button == 5:
                r1_pressed = True
        if event.type == pygame.JOYBUTTONUP:
            # print(event)
            if event.button == 4:
                l1_pressed = False
            if event.button == 5:
                r1_pressed = False
    time.sleep(0.05)
    # end_time = time.time()
    # if (r1_pressed) or (l1_pressed):
    #     print(end_time - start_time)

b'\r\n\x00Done.\r\n>'

SETPVC P1 Z 6572 
SETPVC P1 Z 6572 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Don
MOVEL P1 50
 Done.
>
b''

SETPVC P1 Z 6592 
SETPVC P1 Z 6592 
SETPVC P1 Z 6592 
SETPVC P1 Z 6592 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.

b'>'

SETPVC P1 Z 6612 
SETPVC P1 Z 6612 
SETPVC P1 Z 6612 
SETPVC P1 Z 6612 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Don
MOVEL P1 50
 Done.
>
b''

SETPVC P1 Z 6632 
SETPVC P1 Z 6632 
SETPVC P1 Z 6632 
SETPVC P1 Z 6632 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.
b'\n>'

SETPVC P1 Z 6652 
SETPVC P1 Z 6652 
SETPVC P1 Z 6652 
SETPVC P1 Z 6652 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.
b'\r\n>'

SETPVC P1 Z 6672 
SETPVC P1 Z 6672 
SETPVC P1 Z 6672 
SETPVC P1 Z 6672 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.
b'\r\n>'

SETPVC P1 Z 6692 
SETPVC P1 Z 6692 
SETPVC P1 Z 6692 
SETPVC P1 Z 6692 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.
b'\r\n>'

SETPVC P1 Z 6712 
SETPVC P1 Z 6712 
SETPVC P1 Z 6712 
SETPVC P1 Z 6712 
 Done.
>

MOVEL P1 50
MOVEL P1 50
 Done.
b'\n>'

SETPVC P

KeyboardInterrupt: 

In [ ]:
robot.close()

In [112]:
robot.move_robot_z(-100, 0.05, 0.0)

SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))